#Conway's Game of Life in Reverse

The dataset we'll be working with is available at https://www.kaggle.com/competitions/conways-reverse-game-of-life-2020/data, comprising of 100k games played out, split onto half for training and half for testing. 

The data consists of 0's and 1's placed in a grid of 25x25. The starting grid is randomly initialized. The Game of Life is then run for 5 steps. This allows the introduction of nonlinearity in the evolution. After these 5 steps, the state of the game is recorded. This is then treated as the starting point of the board (called start variables in the dataset). The game is then run for a minimum of one and maximum of five iterations. This number of steps the game evolves into is called the delta value in the data set. After delta steps, the game stops and the state is recorded again (called stop variable in the dataset). If a game dies off in this evolution, the game itself is discarded.

The columns of the dataset are
- game ID (there are 50k of these)
- delta (number of steps)
- start variables (there are 25x25 many of these)
- stop variables (there are 25x25 many of these, none of which are all zeros)

The above dataset gives us our training dataset.

##Stake Holders

Humanity is the stakeholder of this project! The Game of Life is for everyone to appreciate.

##General Model and Key Performance Index
We treat each start variable as our dependent variable $\textbf{y} \in \mathbb{Z}^{25 \times 25}$ and our stop variable as our independent variable $\textbf{x} \in \mathbb{Z}^{1+(25 \times 25)}$. The key performance index is the accuracy of classification, as a ratio of the correctly classified starting configurations against 50,000. That is correctly predicting the entries in the vector $\textbf{y}$. Each $\textbf{y}$ is indexed by the `id` indexing variable in the csv file.

For each $\mathbf{x}$, the first tuple comes from the step $\delta $. The dataset
provides 50,000 such vectors. Our task is to find a function $M\left( 
\mathbf{x}\right) =\mathbf{y}$. Out of the 100,000 $\mathbf{y}$,
we are given a pair of 50,000 such that correspond to the function. Each of
the 50,000 $\mathbf{x}$ evolve from $\mathbf{y}$, depending on delta 
$\delta \in D=\left[ 5\right] =\left\{ 1,2,3,4,5\right\} $

If we treat our problem as that of a classification problem with each $\textbf{x}$ categorized as `id`, a clean up that will be necessary is identifying if any of the starting boards are the same.

We want find optimize $M$ subject to the constraint $$\frac{1}{n}\sum_{i=0}^n \Vert \textbf{y}_i - M(\textbf{x}_i) \Vert^2 $$ where $n=50,000$ and $M$ is the model we want to fit. As a baseline model, we can use Multilinear regression, in which case the above norm should be treated as an $\ell ^2$ norm that needs to be minimized.Treating this problem as a classification problem, we are then bound to the $\ell ^1$ norm. This turns out to be equivalent to the accuracy of classification, which we want to maximise.

The formulation setup in itself assumes that each entry of $\textbf{x}$ is independent of the other, which is already a hurdle. Moreover, we have the problem of treating the delta variable on an equal footing to the inputs or not.

We will split this data set into sets of training $T$, testing $Ts$ and
validation $V$.

##Approach

Can we apply Bayesian classifier? Bayes' Rule tells us that 
$$
P\left( A|B\right) =\frac{P\left( A\cap B\right) }{P\left( B\right) }
$$

In our case, Bayes' Rule translates into $$P\left( \text{starting configuration is }\mathbf{y}|\text{we observe }%
\mathbf{x}\right) =\frac{P\left( \text{starting configuration is }\textbf{y}%
\text{ and we observe }\mathbf{x}\right) }{P\left( \text{we observe }%
\mathbf{x}\right) } 
$$

We let $X$ be a random variable, taking on values $\mathbf{x}$. More
formally, if we let $\Omega =D\times \left\{ 0,1\right\} ^{625}$ be our
sample space, and $X:\Omega \longrightarrow \mathbb{R}$ be given by the rule
that $X\left( \omega \right) $ is the decimal representation of the $5$-ary $%
\omega $, then $X$ has probability mass function given by, for each $i \in \left[625\right]$, $$
f_{X}\left( X\left( \omega \right) \right)_i =\left\{ 
\begin{array}{cc}
p & 
\text{if }\pi _{D}\left( \omega \right)_i =1 \\ 
1-p & \text{if }\pi _{D}\left( \omega \right)_i =0%
\end{array}%
\right. 
$$

Here, $\pi _{D}:D\times \left\{ 0,1\right\} ^{625}\longrightarrow \left\{
0,1\right\} ^{625}$ is the natural projection and the subscript $i$ is the entry for the $i$-th tuple.

In similar vein, we let $Y$ be a random variable, taking on values $\mathbf{y}$. Here, $Y:\left\{ 0,1\right\} ^{625}\longrightarrow \mathbb{R}$ is
given by the decimal representation of $y\in \left\{ 0,1\right\} ^{625}$,
and 
$$
f_{Y}\left( Y\left( y\right) \right) =\left\{ 
\begin{array}{cc}
\frac{\left\vert \mathbf{y}_{i}\right\vert }{\left\vert T\right\vert } & 
\text{if }y=\mathbf{y}_{i}\text{ for some }i \\ 
0 & 0\text{ otherwise}%
\end{array}%
\right. 
$$
Here, $\left\vert \mathbf{y}_{i}\right\vert$ denotes the number of occurances of $\mathbf{y}_{i}$ in $T$. 

We can now write $P\left( Y=\mathbf{y}\right) $ as $P\left( A\right) $,
and $P\left( X=\textbf{x}\right) $ as $P\left( B\right) $, and if we let $%
P\left( \textbf{X}|\mathbf{y}\right) $ be the conditional probability
mass function of $X$ given that the observation came from $\mathbf{y}$.
The alternate form of Bayes' Rule gives us 
$$
P\left( Y=\mathbf{y}|X=X^{\ast }\right) =\frac{P\left( Y=\mathbf{y}\right) P\left( X^{\ast }|\mathbf{y}\right) }{\sum\limits_{j=1}^{%
\left\vert T\right\vert }P\left( Y=\mathbf{y}_{j}\right) P\left( X^{\ast }|%
\mathbf{y}_{j}\right) }
$$

Since we area already making the incorrect assumption that each tuple in $\mathbf{x}$ is independent of another, which simplifies the problem for us, allows us to apply a naive Bayesian classifier. In that case, $P\left( \textbf{x}|\mathbf{y}\right) =\Pi _{i}P\left( x_{i}|\mathbf{y}\right) $. Since the
denominator will remain constant, our task is then to identify where the
numerator of the above equation is maximum. Thus, we want
$$
\arg \max_{\textbf{y}}P\left( Y=\mathbf{y}\right) \prod\limits_{i}P\left(
x_{i}|\mathbf{y}\right) 
$$
We now need to calculate $P\left( Y=\mathbf{y}\right) $ and $P\left(
x_{i}|\mathbf{y}\right) $ and fit the model. We do this by sklearn's built-in functions for Naive Bayesian (Bernoulli) classification. Since the problem is set up for Multinomial Bayesian Classification, Gaussian Naive Bayes Classification and decision trees, we can go ahead and use those as models to compare our Bernoulli Naive Bayesian classification with.

##Importing data and necessary packages

In [ ]:
import pandas as pd
import numpy as np

Working on Google Colab requires us to use the following code to upload our dataset. First we enable uploading on Google Colab

In [ ]:
from google.colab import files
uploaded = files.upload()
#allows us to upload files on colab.

Saving gameoflife_train.csv to gameoflife_train.csv


In [ ]:
#allows us to read_csv file
import io

#game_of_life_test_df = pd.read_csv(io.BytesIO(uploaded['gameoflife_test.csv']))
game_of_life_train_df = pd.read_csv(io.BytesIO(uploaded['gameoflife_train.csv']))
#files can be downloaded from https://www.kaggle.com/competitions/conways-reverse-game-of-life-2020/data


On a local version of the notebook, a simple upload suffices (though the path will need to be updated)

In [ ]:
game_of_life_train_df = pd.read_csvs('gameoflife_train.csv')

In either case, we can now view our uploaded dataset

In [ ]:
#check to see if data has been loaded
game_of_life_train_df

,id,delta,start_0,start_1,start_2,start_3,start_4,start_5,start_6,start_7,...,stop_615,stop_616,stop_617,stop_618,stop_619,stop_620,stop_621,stop_622,stop_623,stop_624
0,0,3,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,3,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,3,5,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,1,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,4,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
49996,49996,4,0,0,0,0,1,1,0,1,...,0,0,0,0,1,1,0,0,0,0
49997,49997,2,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,1,0
49998,49998,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##Data cleaning

We will train the model using two approaches: one would be by discarding all delta values and treating only the ending values of the board as our starting configuration, as mentioned above. 

In [ ]:
#use only starting values as independent variable. In particular, we ignore delta values
game_of_life_train_df_X_no_delta = game_of_life_train_df.iloc[:, 2:627]
#check output
game_of_life_train_df_X_no_delta

,start_0,start_1,start_2,start_3,start_4,start_5,start_6,start_7,start_8,start_9,...,start_615,start_616,start_617,start_618,start_619,start_620,start_621,start_622,start_623,start_624
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,1,...,0,0,1,0,1,1,1,0,1,1
2,0,1,0,0,0,0,0,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,0,0,0,0,0,...,1,1,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,1,1,0,1,...,0,0,1,0,1,1,0,0,0,1
49996,0,0,0,0,1,1,0,1,1,1,...,1,0,0,1,1,0,0,0,0,0
49997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,1
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Another approach would treat the values of the delta parameters on an equal footing to that of the grid entries. To do this, we'll have to make sure that the delta values are stored as binary strings in a new panda frame.

In [ ]:
#extract only column of delta-values as a numpy array.
delta_values = game_of_life_train_df.iloc[:, 1].to_numpy()
#create empty numpy array to hold delta values in binary form (as strings)
delta_values_in_binary = np.empty(delta_values.shape, dtype='U3')

# Convert each entry to binary representation with three digits
for i, num in enumerate(delta_values):
    binary_repr = '{:03b}'.format(num)
    delta_values_in_binary[i] = binary_repr
#change the array to split each 3-digit binary array into a row with 3 columns
delta_values_in_binary = np.vstack([list(binary) for binary in delta_values_in_binary])
#use the resulting array to make a panda dataframe
game_of_life_train_df_deltas_in_binary = pd.DataFrame(
    delta_values_in_binary, columns=[
        'Delta_digit_2', 'Delta_digit_1', 'Delta_digit_0'])
#concatenate this newly created dataframe with only stopping values
game_of_life_train_df_X_and_deltas_in_binary = pd.concat([game_of_life_train_df_deltas_in_binary, game_of_life_train_df_X_no_delta], axis=1)
#see outcome
game_of_life_train_df_X_and_deltas_in_binary

,Delta_digit_2,Delta_digit_1,Delta_digit_0,start_0,start_1,start_2,start_3,start_4,start_5,start_6,...,start_615,start_616,start_617,start_618,start_619,start_620,start_621,start_622,start_623,start_624
0,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,1,1,0,1,1
2,0,1,1,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,1,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,1,0,0,...,1,1,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,0,0,0,0,0,0,0,0,1,...,0,0,1,0,1,1,0,0,0,1
49996,1,0,0,0,0,0,0,1,1,0,...,1,0,0,1,1,0,0,0,0,0
49997,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,1
49998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#use stopping values as dependent variable
game_of_life_train_df_Y = game_of_life_train_df.iloc[:, 627:]

##Using regression
For fun..

In [ ]:
#convert the panda dataframe to a numpy object for implementation with sklearn regression
X_train_regression = game_of_life_train_df.iloc[:, 1:627].to_numpy()
#check output
X_train_regression

array([[3, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 1, 1],
       [3, 0, 1, ..., 0, 0, 0],
       ...,
       [2, 1, 0, ..., 1, 1, 1],
       [4, 0, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 1, 0, 0]])

In [ ]:
#To use simple regression, we will need the following y_values
Y_train = game_of_life_train_df_Y.values
#replace column of delta to 1s for beta_0 for regression
X_train_regression[:,0] = np.ones((50000))
#check output
X_train_regression

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 1],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 0, 0]])

The baseline model we have is $\hat{\beta}\textbf{x} + \epsilon=\textbf{y}$

In [ ]:
## Use sklearn to fit the baseline model LinearRegression object
from sklearn.linear_model import LinearRegression
reg = LinearRegression(copy_X=True, fit_intercept=False)
reg.fit(X_train_regression, Y_train)
reg.coef_

array([[ 0.00839247,  0.16294614,  0.03920627, ..., -0.00461023,
         0.01357494,  0.04310577],
       [ 0.00757545,  0.05134265,  0.15249852, ..., -0.00479549,
         0.00591691,  0.01363778],
       [ 0.00743363,  0.01878222,  0.04793412, ...,  0.0051466 ,
        -0.0024245 ,  0.00408084],
       ...,
       [ 0.00974763,  0.00271165,  0.00568008, ...,  0.14278846,
         0.05001605,  0.01677448],
       [ 0.01007119,  0.01142097,  0.01160265, ...,  0.04743408,
         0.14383655,  0.04411862],
       [ 0.00863377,  0.03392907,  0.02734676, ...,  0.01145433,
         0.05336593,  0.14243488]])

In [ ]:
y_pred_sklearn = reg.predict(X_train_regression)
mse = np.sum(np.power(Y_train-y_pred_sklearn, 2))/len(Y_train)
print("the mse is", mse)

the mse is 65.23835237230107


Remember, the above error is for $\ell^2$ norm! To make a meaningful comparison with another $\ell^2$ norm, another baseline model we could go would be a simple polynomial in each $x_i$, with 625 independent variables, and degree depending upon the delta. Let us begin with a 5 degree polynomial
$P\left( \mathbf{x}\right)
=\sum%
\limits_{i_{k}}c_{i_{1}i_{2}i_{3}i_{4}i_{5}}x_{i_{1}}x_{i_{2}}x_{i_{3}}x_{i_{4}}x_{i_{5}}
$ where $0\leq i_{1}\leq i_{2}\leq i_{3}\leq i_{4}\leq i_{5}\leq 650$ and try to fit this in, but even colab gives up.



In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(5,
                             interaction_only = False,
                             include_bias = True)
poly.fit_transform(X_train_regression, Y_train)
poly.coef_

##Using Naive Bayes and more data cleaning

Since the above code fails :( we can take comfort in the fact that this approach was only for comparison with regression, so we can safely drop this approach and try our real approach with classification. We start cleaning the data now to remove multiple entries for $\textbf{y}$. If you run the code below, you'll see that our instances of $\textbf{y}$ reduce to 48966, instead of 50000.

In [ ]:
game_of_life_train_df_Y_noduplicates=game_of_life_train_df_Y.drop_duplicates()
game_of_life_train_df_Y_noduplicates

,stop_0,stop_1,stop_2,stop_3,stop_4,stop_5,stop_6,stop_7,stop_8,stop_9,...,stop_615,stop_616,stop_617,stop_618,stop_619,stop_620,stop_621,stop_622,stop_623,stop_624
0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
49996,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
49997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,1,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#create a dataframe that comes from only duplicate values
gol_y = game_of_life_train_df_Y[game_of_life_train_df_Y.duplicated(keep=False)]
#Using this list, create a list of lists, where each sublist contains indices where y indices are repeated 
game_of_life_train_df_duplicates_groups = gol_y.groupby(list(gol_y)).apply(lambda x: list(x.index)).tolist()
game_of_life_train_df_duplicates_groups
#Now we create an array only containing indices which are repeated for similar y values.

categories = np.arange(0, 50000)
for group in game_of_life_train_df_duplicates_groups:
    for index in group:
        categories[categories == index] = group[0]

For both models, we now have a training set $[\textbf{x}_0, \textbf{x}_1, ..., \textbf{x}_{49999}]$ and a 48,966-long list of categories. This approach is inefficient, because there are too many categories!

To limit the number of categories, what we can do is mod out the space for $Y
$ by treating $\mathbf{y}_{1}$ and $\mathbf{y}_{2}$ if they are close
enough, like a modification of a $k$-nearest neighbor approach.

What metric do we use? A tempting metric here would be the Hamming distance:
$
d\left( \mathbf{y}_{1},\mathbf{y}_{2}\right) =\left\vert \mathbf{y}_{1}+%
\mathbf{y}_{2}\right\vert 
$
where addition here is tuple-wise, and $\left\vert .\right\vert $ counts the
number of $1$ entries we get. This is effectively checking where two
(binary) vectors $\mathbf{y}_{1}$ and $\mathbf{y}_{2}$ differ. Thus, $0\leq
d\left( \mathbf{y}_{1},\mathbf{y}_{2}\right) \leq 625$ in our case.

The data cleaning has already done this for the case when $\mathbf{y}_{1}=%
\mathbf{y}_{2}$. We want to make the additional assumption that if $d\left( 
\mathbf{y}_{1},\mathbf{y}_{2}\right) =k$, then we force $\mathbf{y}%
_{1}\simeq \mathbf{y}_{2}$. We have to be careful here to make sure that
this equality stays locally true. This is because the triangle equality
will break transitivity of our sought-after equivalence relation. For
example, $d\left( 0000,1100\right) =2=d\left( 1111,1100\right) $, but $d\left( 1111,0000\right) =4$. To that end, we follow [1], and we say that $%
\mathbf{y}_{1}$ and $\mathbf{y}_{2}$ are $p$-close if any subsequence of
length $p\leq 625$ comprising of consecutive $p$ grid values appears the
same number of times (which might be also zero) both in $\mathbf{y}_{1}$ and 
$\mathbf{y}_{2}$. If this holds, then we write $\mathbf{y}_{1} \overset{p}{\simeq }\mathbf{y}_{2}$. Clearly, this relation is reflexive,
symmetric and transitive.

We pick $p=9$, since that comprises of one neighborhood of
a cell, including itself. This has the added advantage of allowing us to only check against $2^9$ possible subsequences between $\mathbf{y}_{1}$ and $\mathbf{y}_{2}$.

Therefore, we will then have to fit a Bernoulli Naive Classifier into our given data $\textbf{x}$ against elements (classes) of the space $\mathcal{Y}=Y/\simeq$. For each class $[\textbf{y}]$ predicted, we can then run each element in $[\textbf{y}]$ and see which one matches our given $\textbf{x}$, based off of the given value of $\delta$. This check is implemented below:

[1] Boris Gutkin and Vladimir Al Osipov 2013 _Nonlinearity_ $\textbf{26
}$ 177


In [ ]:
import itertools
from itertools import combinations

#create all combinations of vectors y_1 and y_2
combinations_rows = [game_of_life_train_df.iloc[:, 627:].to_numpy()[list(combo)] for combo in combinations(range(game_of_life_train_df.iloc[:, 627:].to_numpy().shape[0]), 2)]

combinations_rows

#create empty container for combinations to be used to create above array
combos=[]
#create combinations
for combination in itertools.product([0, 1], repeat=9):
    combos.append(combination)

# Convert the list of combinations to array
checkers = np.array(combos)

#create list to contain pairs that are p-close
pairs = []

#see if every possible y1,y2 have every possible combination above as a subsequence in both same number of times
for combo in combinations_rows:
    a_bool = False
    arr1 = np.copy(combo[0])
    arr2 = np.copy(combo[1])
    view1 = np.lib.stride_tricks.sliding_window_view(arr1, window_shape=(9,))
    view2 = np.lib.stride_tricks.sliding_window_view(arr2, window_shape=(9,))
    for k in len(range(0,513)):
        array3 = np.copy(checkers[k,:])
        
    # Compare views with array3 and count the number of matches
        if np.sum(np.all(view1 == array3, axis=1)) == np.sum(np.all(view2 == array3, axis=1)):
            a_bool = True
        else:
            a_bool = False
            break
    if a_bool:
        pairs.append([arr1,arr2])

#duplicate category labels based off of how p-close they are
for group in game_of_life_train_df_duplicates_groups:
    for index in group:
        categories[categories == index] = group[0]

##Final implementation

Since each $\textbf{x}$ has binary variables, we use Bernoulli Naive Bayesian below. Since we have the code set up, why not check for the other distributions, too, in the same go? We will also create a dataframe that will allow us to compare all models.

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

trainingSet0 = game_of_life_train_df_X_no_delta.to_numpy()
trainingSet1 = game_of_life_train_df_X_and_deltas_in_binary.to_numpy()
testingSet = categories
#load models
bnb = BernoulliNB()
gnb = GaussianNB()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
model0bnb = bnb.fit(trainingSet0, testingSet)
model1bnb = bnb.fit(trainingSet1, testingSet)
model0gnb = gnb.fit(trainingSet0, testingSet)
model1gnb = gnb.fit(trainingSet1, testingSet)
model0mnb = mnb.fit(trainingSet0, testingSet)
model1mnb = mnb.fit(trainingSet1, testingSet)
model0dtc = dtc.fit(trainingSet0, testingSet)
model1dtc = dtc.fit(trainingSet1, testingSet)

# Create a dataframe to compare performance of Classifier Models
classifiers_compare = pd.DataFrame()

#cross-validation scores with k=5 plus adding to dataframe
scores0bnb = cross_val_score(bnb, trainingSet0, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["BernoulliNB_0", (scores0bnb.mean())]])

scores1bnb = cross_val_score(bnb, trainingSet1, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["BernoulliNB_1", (scores1bnb.mean())]])

scores0gnb = cross_val_score(gnb, trainingSet0, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["GaussianNB_0", (scores0gnb.mean())]])

scores1gnb = cross_val_score(gnb, trainingSet1, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["GaussianNB_1", (scores1gnb.mean())]])

scores0mnb = cross_val_score(mnb, trainingSet0, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["MultinomialNB_0", (scores0mnb.mean())]])

scores1mnb = cross_val_score(mnb, trainingSet1, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["MultinomialNB_1", (scores1mnb.mean())]])

scores0dtc = cross_val_score(dtc, trainingSet0, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["DecisionTree_0", (scores0dtc.mean())]])

scores1dtc = cross_val_score(dtc, trainingSet1, testingSet, scoring="accuracy")
classifiers_compare = classifiers_compare.append([["DecisionTree_1", (scores0dtc.mean())]])

classifiers_compare = classifiers_compare.append([["Regression", (mse)]])

#plot of mean scores for each version:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
sns.set(style="whitegrid")
sns.barplot(x=0, y=1, data=classifiers_compare, palette="BuGn_d")
plt.xlabel("Model and Distribution")
plt.ylim(0,1)
plt.ylabel("Percent Accuracy")
plt.show()